# Korea Datathon Tutorial

The aim of this tutorial is to get you familarized with using Colab to run queries with MIMIC database. 

### environment
* python3.6
* tensorflow 2.0(beta)
* GCP(bigquery)

### Prerequisites

* SQL(basic)
* python3(basic)
* statistics(hypothesis testing)
* machine-learning(regresison, multi-perceptron)

## Setup

* Before running any cell in the script, please make sure that there is a green check mark before "CONNECTED" on top right corner, if not, please click "CONNECTED" button to connect to a random backend.

* You can now run the following cell by clicking on the triangle button when you hover over the [ ] space on the top-left corner of the code cell below or Press Shift+Enter.s

In [ ]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm


from sklearn import linear_model
from itertools import cycle, islice
from regressors import stats
from toolz import *
from matplotlib import pyplot as plt
from IPython.display import display, HTML
from google.colab import auth
from google.cloud import bigquery

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

* Before running any queries using BigQuery, you need to first authenticate yourself by running the following cell. If you are running it for the first time, it will ask you to follow a link to log in using your Gmail account, and accept the data access requests to your profile. Once this is done, it will generate a string of verification code, which you should paste back to the cell below and press enter." 

In [ ]:
auth.authenticate_user()

* The data-hosting project physionet-data has read-only access, as a result, you need to set a default project that you have BigQuery access to.

In [ ]:
project_id= "kor-data" #
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

In [ ]:
# a wrapper function that presets some configs
def run_query(query):
    return pd.io.gbq.read_gbq(query, 
                              dialect="legacy",
                              project_id     = project_id,
                              configuration = {'query':{'useLegacySql': False}})

## sample queries

1. Select all columns from a **`patients`** table where **subject_id** is "10006 | 10040 | 10111".

In [ ]:
df = run_query(
   """
    select * 

    from `{db}.patients` 
    
    where 
        subject_id = 10006 OR
        subject_id = 10040 OR
        subject_id = 10111
    """\
    .format(db = "physionet-data.mimiciii_demo"))

df.head()

In [ ]:
df_1 = run_query(
    """
    select p.subject_id, a.hadm_id, icustay_id, dob, admittime, dischtime, intime, outtime, ethnicity, gender

    from `{db}.patients` p, `{db}.admissions` a, `{db}.icustays` i

    where p.subject_id = a.subject_id AND 
          a.hadm_id    = i.hadm_id
    """\
    .format(db = "physionet-data.mimiciii_demo"))

# or...


df_2 = run_query(
    """ 
    select p.subject_id, a.hadm_id, icustay_id, dob, admittime, dischtime, intime, outtime, ethnicity, gender  
    from `{db}.patients` p
    inner join `{db}.admissions` a USING (subject_id)
    inner join `{db}.icustays` i USING (hadm_id)
    """\
    .format(db = "physionet-data.mimiciii_demo")
)


def is_same(df_1, df_2):
    if all(df_1 == df_2): return "the two dfs are equal"
    else: return "the two dfs are not equal"
      
print(is_same(df_1, df_2))
print("\n")

df_1.head()

In [ ]:
##1
df = run_query("""
    select icustay_id, max(valuenum) as HeartRate_Max
    from `{db}.chartevents`
    where itemid in (
    
        select itemid
        from `{db}.d_items`
        where lower(label) = 'heart rate'
                    )
    group by icustay_id
      """\
    .format(db = "physionet-data.mimiciii_demo"))
df.head()

In [ ]:
##2 
df = run_query("""
    select icustay_id, max(valuenum) as HeartRate_Max
    from `{db}.chartevents`
    where itemid in (
    
        select itemid
        from `{db}.d_items`
        where lower(label) = 'heart rate'
                    )
    group by icustay_id
        having HeartRate_Max <= 140
      """\
    .format(db = "physionet-data.mimiciii_demo"))
df.head()

It is important to note that patients in mimiciii data with their **age > 91.5 are masked by shifting thier DOB**. Therefore, **threshold of 91.5 has to be set or simply ignore them in your study**.

1. Get the list of subject_id, patient_id with thier primary diagnoses ICD code and their age in years.
2. And only take thier first admit

In [ ]:
df = run_query("""

     -- first_hadm_id table will store subject_id, hadm_id of patient's first visit.
    with first_hadm_id as (
    
      select subject_id, hadm_id
      from(
        select subject_id, hadm_id, row_number() over (partition by subject_id order by admittime asc) row_num
        from `{db}.admissions` 
        ) t  
      where row_num = 1 
      ),
      
    -- modified_age if age < 91.5 then age else 91.5
    modified_age as 
    (
        select subject_id, hadm_id, 
            case 
            when date_diff(cast(admittime as date),cast(dob as date),day)/365 < 91.5
                then date_diff(cast(admittime as date),cast(dob as date),day)/365
            else 91.5
            end as age 
        from `{db}.admissions` a 
        left join `{db}.patients` p
          using (subject_id)
    ),
    
    -- get primary diagnosis of each patient visit.
    primary_diag as
    (
        select subject_id, hadm_id,
          (select d_d.short_title
           from `{db}.d_icd_diagnoses` d_d
           where d_d.icd9_code = d.icd9_code
          ) as diagnosis
          
        from `{db}.diagnoses_icd` d
        where seq_num = 1
    )
    
    -- merge all sub queries with columns (subject_id, hadm_id, modified_age,primary_diag)    
    select first_hadm_id.*, age, diagnosis
    from primary_diag
    left join modified_age using(subject_id, hadm_id)
    left join first_hadm_id using(subject_id, hadm_id)
    
"""\
.format(db = "physionet-data.mimiciii_demo"))

df.head()

## General Workflow 

In general, data extraction from MIMICiii data can be divived in to several steps. tables created from each step can be merged to generate a bigger table which will be used for our final analysis. 

1. Cohort selection + demographics 
2. Outcome of interest
2. Diagnosis
3. Labtest results
4. VitalSign
5. Merge 

## CASE STUDY: Determinants of length of stay 
Lets make a dummy cohort. We will observe what covaraites are responsible for their lenght of stay.

1. Cohort selection + demographics : entire cohort(first admit), age(year), gender(M|F).
2. Outcome of interest : length of stay(days), length of stay(long|short).
3. Diagnosis : number of diagnosis.
4. Labtest results: White_blood_cell_mean(24H), Hemoglobin_mean(24H)
5. VitalSign: heart_rate_mean(24H), Temperature_max(24H), Temperature_min(24H)
6. Merge 

In [ ]:
# Lets create a dictionary object to store queries of each.
queries = \
    {"cohort_demo" : None, # entire cohort(first admit), age(year), gender(M|F).
     "outcome"     : None, # length of stay(days), length of stay(long|short).
     "diagnosis"   : None, # number of diagnosis.
     "lab"         : None, #  White_blood_cell_mean(24H), Hemoglobin_mean(24H)
     "vital"       : None} #  heart_rate_mean(24H), Temperature_max(24H), Temperature_min(24H) 

In [ ]:
queries["cohort_demo"] = \
"""
    -- first_hadm_id table will store subject_id, hadm_id of patient's first visit.
    with first_hadm_id as (
    
      select subject_id, hadm_id
      from(
        select subject_id, hadm_id, row_number() over (partition by subject_id order by admittime asc) row_num
        from `{db}.admissions` 
        ) t  
      where row_num = 1 
      ),
      
    -- modified_age if age < 91.5 then age else 19.5
    modified_age as 
    (
        select subject_id, hadm_id, 
            case 
            when date_diff(cast(admittime as date),cast(dob as date),day)/365 < 91.5
                then date_diff(cast(admittime as date),cast(dob as date),day)/365
            else 91.5
            end as age 
        from `{db}.admissions` a 
        left join `{db}.patients` p
          using (subject_id)
    ),
    
    -- gender. if M then 1 else 0 
    
    gender as 
    (
        select subject_id, 
            case when gender = 'M' then 1 else 0 end as sex
        from `{db}.patients`
    )
    
    -- merge all queries
    select first_hadm_id.*, age, sex
    from first_hadm_id
    left join modified_age using(subject_id, hadm_id)
    left join gender using(subject_id)
    
"""\
.format(db = "physionet-data.mimiciii_demo")


### 2. Outcome of interest : length of stay(days), length of stay(long|short). ###


In [ ]:
queries["outcome"] = \
"""
with length_of_stay as (
    select subject_id, hadm_id, icustay_id, 
        datetime_diff(icu.outtime, icu.intime, hour) AS icu_length_of_stay,
        row_number() over (partition by subject_id order by icu.intime asc) row_num
    from `{db}.icustays` as icu)
    
select subject_id, hadm_id, icu_length_of_stay as stay_con, 
       case 
       when icu_length_of_stay > 100  then 1
       else 0 end as stay_bi
from length_of_stay
where row_num = 1
"""\
.format(db = "physionet-data.mimiciii_demo")

### 3. Diagnosis : number of diagnosis. ###

In [ ]:
queries["diagnosis"] = \
"""
select hadm_id, count(icd9_code) as n_diag
from `{db}.diagnoses_icd`
group by hadm_id
"""\
.format(db = "physionet-data.mimiciii_demo")

### 4. Labtest results: White_blood_cell_mean(24H), Hemoglobin_mean(24H). ###


In [ ]:
queries["lab"] = \
"""
with lab_tests as (
    select hadm_id, charttime, value, valuenum,
           case when itemid in (50811,51222) then 'hemoglobin'
                when itemid in (50822,50971) then 'potassium'
                else null end as label                     
    from `{db}.labevents`),
    
lab_tests_prune as (
    select subject_id, hadm_id, icustay_id, label, valuenum, charttime 
    from `{db}.icustays` 
    left join lab_tests lab using (hadm_id)
    where charttime between intime and datetime_add(intime,interval 1 day)
          and label is not null
          and valuenum is not null
          and valuenum > 0)

select hadm_id, icustay_id,
   avg(case when label = 'hemoglobin' then  valuenum else null end) as hemoglobin_avg,
   avg(case when label = 'potassium'  then  valuenum else null end) as troponin_avg
from lab_tests_prune
group by hadm_id, icustay_id
"""\
.format(db = "physionet-data.mimiciii_demo")

### 5. VitalSign: Heart_Rate_mean(24H), Temperature_max(24H), Temperature_min(24H). ###

In [ ]:
queries["vital"] = \
"""
with vital_tests as (
    select hadm_id, charttime, value, valuenum,
           case when itemid in (223762,676,223761,678) then 'temperature'
                when itemid in (211,220045) then 'blood_pressure'
                else null end as label                     
    from `{db}.chartevents`),
    
vital_tests_prune as (
    select subject_id, hadm_id, icustay_id, label, valuenum, charttime 
    from `{db}.icustays` 
    left join vital_tests lab using (hadm_id)
    where charttime between intime and datetime_add(intime,interval 1 day)
          and label is not null
          and valuenum is not null
          and valuenum > 0)

select hadm_id, icustay_id,
   max(case when label = 'temperature' then  valuenum else null end) as temperature_max,
   min(case when label = 'temperature' then  valuenum else null end) as temperature_min,
   avg(case when label = 'blood_pressure' then   valuenum else null end) as blood_pressure_mean
from vital_tests_prune
group by hadm_id, icustay_id
"""\
.format(db = "physionet-data.mimiciii_demo")

### 6. Merge. ###

In [ ]:
tables = map(run_query, queries.values())
df = reduce(lambda x,y : pd.merge(x,y), tables)

In [ ]:
df.head()

## Linear Regression for correlation analysis

We may want to find the correlation between some input variables and outcome varialbe.

In [ ]:
# which column contains na?
na_columns = \
[(c_name, df[c_name].dtypes)
     for c_name 
     in df.columns
     if any(pd.isnull(df[c_name]))]

print(na_columns)

# maen imputation
for na_col in map(lambda x : x[0],na_columns):
    df[na_col] = df[na_col].fillna((df[na_col].mean()))
    
    
# select columns of interest
X_name = ['hemoglobin_avg',
          'n_diag',
          'troponin_avg',
          'age',
          'sex',
          'temperature_min',
          'blood_pressure_mean',
          'temperature_max',]

Y_name = "stay_con"

X = df[X_name]; y = df[Y_name]
# Linear regression
ols = linear_model.LinearRegression()
ols.fit(X,y)

# to print summary table:
print("\n=========== SUMMARY ===========")
xlabels = X_name
stats.summary(ols, X, y, xlabels)

## Logistic Regression for correlation analysis

We may want to find the correlation between some input variables and outcome varialbe.

In [ ]:
# select columns of interest
X_name = ['hemoglobin_avg',
          'n_diag',
          'troponin_avg',
          'age',
          'sex',
          'temperature_min',
          'blood_pressure_mean',
          'temperature_max',]

Y_name = "stay_bi"

X  = df[X_name]; y = df[Y_name]
# Linear regression
logit = sm.Logit(y,X)
# fit the model
result = logit.fit()
# to print summary table:
print("\n=========== SUMMARY ===========")
print(result.summary())

## Nerual Net for prediction

We may want to come out with a prediction model

In [ ]:
class config():
    input_size    = 8  
    train_len     = 80    
    output_size   = 2 
    learning_rate = 10**-5
    batch_size    = 2
    epoch         = 2
    
    
"""
preprocessing tools
""" 
set_format       = np.float32
normalise        = lambda xs : (xs - np.mean(xs))/np.var(xs)
add_random_noise = lambda xs : xs + np.random.normal(0,0.001,len(xs)) 

preprocess = compose(add_random_noise,normalise,set_format)

"""
data control tools
"""
data_gen = lambda X : list(map(list,zip(*([X] * config.batch_size))))

"""
model
"""
model = tf.keras.Sequential([
  tf.keras.layers.Dense(12, activation=tf.nn.relu, input_shape=(config.input_size,)),  # input shape required
  tf.keras.layers.Dense(24, activation=tf.nn.relu),
  tf.keras.layers.Dense(2, activation = tf.nn.sigmoid)
])

def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    return loss_value, tape.gradient(loss_value, model.weights)


loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
def loss(model, x, y):
    y_ = model(x)
    return loss_object(y_true=y, y_pred=y_)

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001)

In [ ]:
X_val = X.values
y_val = y.values

X_train, y_train = X_val[:config.train_len], y_val[:config.train_len]
X_test,  y_test  = X_val[config.train_len:], y_val[config.train_len:] 

In [ ]:
## Note: Rerunning this cell uses the same model variables

# Keep results for plotting
train_loss_results = []
train_accuracy_results = []

for epoch in range(config.epoch):

    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

    # Training loop - using batches of 32
    for _x, _y in zip(data_gen(map(preprocess, X_train)),
                      data_gen(y_train)):
        
        x_np = np.array(_x)
        y_np = np.array(_y)
        
        # Optimize the model
        loss_value, grads = grad(model, x_np, y_np)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Track progress
        epoch_loss_avg(loss_value)  # Add current batch loss
        # Compare predicted label to actual label
        epoch_accuracy(y_np, model(x_np))

    # End epoch
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())

    if epoch % 1 == 0:
      print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                  epoch_loss_avg.result(),
                                                                  epoch_accuracy.result()))